In [1]:
import numpy as np
from torch import nn
import torch.nn.functional as F
import torch
from preprocessing import *
from skorch import NeuralNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.utils import class_weight


/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [107]:
cw = list(class_weight.compute_class_weight('balanced',
                                             [c for c in range(len(encoder.classes_))],
                                             y_train))

In [108]:
cw = torch.tensor(cw)

In [44]:
cw.shape

torch.Size([7])

In [2]:
X, y, encoder = preprocess_data('TOTAL_TRAIN.csv', process_cat=True)

/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (54) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Valores unicos de CADASTRAL--- ['2' '3' '4' '8' '5' '6' '7' '9' '1' 'C' 'B' 'A' 7 6 3 4 5 8 9 2 1 nan]
En momento 2 el shape es de (103230, 141)
Las columnas que tienen dtype object son Index(['ruido', 'cluster'], dtype='object')
En momento 3 el shape es de (103230, 243)
Imputando valores con Random Forest
Iteration: 0
Iteration: 1
Iteration: 2
En momento 4 el shape es de (103230, 243)
En momento 5 el shape es de (103230, 253)
En momento 6 el shape es de (103230, 253)


In [3]:
X = X.astype('float')
y = y.astype('int')

In [4]:
y.shape

(103230,)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.15)

In [147]:
class MyModule(nn.Module):
    def __init__(self, units1=512, units2 = 512, units3=512, units4=512, drop1 = 0.4,
                 drop2 = 0.4, drop3 = 0.4, drop4=0.4, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(X_train.shape[1], units1)
        self.nonlin = nonlin
        self.drop1 = nn.Dropout(drop1)
        self.dense1 = nn.Linear(units1, units2)
        self.drop2 = nn.Dropout(drop2)
        self.dense2 = nn.Linear(units2, units3)
        self.drop3 = nn.Dropout(drop3)
        self.dense3 = nn.Linear(units3, units4)
        self.drop4 = nn.Dropout(drop4)
        self.output = nn.Linear(units4, 7)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.drop1(X)
        X = F.relu(self.dense1(X))
        X = self.drop2(X)
        X = F.relu(self.dense2(X))
        X = self.drop3(X)
        X = F.relu(self.dense3(X))
        X = self.drop4(X)
        X = nn.LogSoftmax(X)#F.softmax(self.output(X), dim=-1)
        return X

In [148]:
from skorch.callbacks import LRScheduler, GradientNormClipping

In [149]:
scheduler = LRScheduler(monitor='val_loss')
clip_grad = GradientNormClipping(1)

In [150]:
cw = list(class_weight.compute_class_weight('balanced',
                                             [c for c in range(len(encoder.classes_))],
                                             y_train))

In [151]:
cw = torch.tensor(cw, device='cuda')

In [152]:
class NLL(torch.nn.modules.loss._Loss):
    def forward(self, inp, target):
        global cw
        loss = F.nll_loss(inp, target, weight=cw)
        return loss 

In [153]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=30,
    lr=0.001,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    criterion=NLL,
    device='cuda',
    batch_size=256, 
    optimizer=torch.optim.Adam,
    callbacks=[clip_grad]
)

In [154]:
~net.fit(torch.tensor(np.array(X_train)).float(), torch.tensor(y_train))

TypeError: 'Tensor' object is not callable

In [132]:
preds = net.predict(torch.tensor(X_test.values).float())

In [133]:
f1_score(y_test, preds, average='macro')

0.20881087953877822

In [ ]:
tr_latent_X = data_utils.TensorDataset(train_x, train_y)
te_latent_X = data_utils.TensorDataset(test_x,  test_y)